<font color='blue' size=5><b>Комментарий ревьюера</b></font>

<font color='blue'>Привет, Александр! Меня зовут Павел Григорьев, и я буду проверять этот проект.<br>Моя основная цель - не указать на совершённые тобой ошибки, а поделиться своим опытом и помочь тебе совершенствоваться как профессионалу.<br>Спасибо за проделанную работу! Предлагаю общаться на «ты».</font>
<details>
	<summary><u>Инструкция по организационным моментам (кликабельно)</u></summary>
<font color='blue'>Я буду использовать различные цвета, чтобы было удобнее воспринимать мои комментарии:</font>


---


<font color='blue'>синий текст - просто текст комментария</font>

<font color='green'>✔️ и зеленый текст - все отлично</font>

<font color='orange'>⚠️ и оранжевый текст - сделано все правильно, однако есть рекомендации, на что стоит обратить внимание</font>

<font color='red'>❌ и красный текст - есть недочеты</font>


</details>    
    </br>
<font color='blue'>Пожалуйста, не удаляй мои комментарии в случае возврата работы, так будет проще разобраться, какие были недочеты, а также сразу увидеть исправленное. </font>

Ответы на мои комментарии лучше тоже помечать.
Например: <font color='purple'><b>Комментарий студента</b></font>

<font color='blue'><b>Давай смотреть, что получилось!</b></font>

## Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

# Подготовка данных

In [1]:
import pandas as pd 
import numpy as np

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

pyspark_version = pyspark.__version__
if int(pyspark_version[:1]) == 3:
    from pyspark.ml.feature import OneHotEncoder    
elif int(pyspark_version[:1]) == 2:
    from pyspark.ml.feature import OneHotEncodeEstimator

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Хорошее оформление импортов! \
Импорты собраны в одной ячейке, разделены на функциональные группы пустой строкой.</font>

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'>Здорово, что есть условие по версии pyspark. \
Можно ещё сразу положить Энкодер в переменную, чтобы далее обращаться только к ней. </font>

In [2]:
RANDOM_SEED = 12321

In [3]:
spark = (SparkSession.builder
                    .master("local")
                    .appName("Housing - Logistic regression")
                    .getOrCreate())

df = spark.read.option('header', 'true').csv('/datasets/housing.csv', inferSchema = True) 

In [4]:
for column in df.columns:
    check_col = F.col(column).cast(FloatType()).isNull()
    print(column, df.filter(check_col).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 207
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 20640


<font color='blue'><b>Комментарий ревьюера: </b></font> ⚠️\
<font color='darkorange'> Нет пропусков?</font>

<font color='purple'><b>Комментарий студента</b></font>\
<font color=#4B0082><b>Вот вот же они)</b></font>

In [5]:
df = df.na.fill(535, ['total_bedrooms'])
df = df.na.fill('Unknown', ['ocean_proximity'])

In [6]:
df.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20640,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8417635658915,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,419.2666898951927,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


<font color='blue'><b>Комментарий ревьюера: </b></font> ❌\
<font color='red'>По заданию пропуски нужно именно заполнить, выбрав значения по своему усмотрению.</font>

<font color='purple'><b>Комментарий студента</b></font>\
<font color=#4B0082><b>Заполнил пропуски</b></font>

<font color='blue'><b>Комментарий ревьюера: </b></font> ❌\
<font color='red'>При использовании .toPandas() ты выгружаешь данные в оперативную память, что при работе с действительно большими данными приведет к переполнению памяти, этот метод применяется только к агрегированным таблицам(либо к ограниченным подвыборкам). Попробуй реализовать схожий функционал с применением методов pySpark.
</font>

<font color='purple'><b>Комментарий студента</b></font>\
<font color=#4B0082><b>Починил</b></font>

In [7]:
df.dtypes

[('longitude', 'double'),
 ('latitude', 'double'),
 ('housing_median_age', 'double'),
 ('total_rooms', 'double'),
 ('total_bedrooms', 'double'),
 ('population', 'double'),
 ('households', 'double'),
 ('median_income', 'double'),
 ('median_house_value', 'double'),
 ('ocean_proximity', 'string')]

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Отлично!  Выведены типы данных.</font>

In [8]:
df = df.withColumn('longitude', F.col('longitude').cast(FloatType()))
df = df.withColumn('latitude', F.col('latitude').cast(FloatType()))
df = df.withColumn('housing_median_age', F.col('housing_median_age').cast(IntegerType()))
df = df.withColumn('total_rooms', F.col('total_rooms').cast(IntegerType()))
df = df.withColumn('total_bedrooms', F.col('total_bedrooms').cast(IntegerType()))
df = df.withColumn('population', F.col('population').cast(IntegerType()))
df = df.withColumn('households', F.col('households').cast(IntegerType()))
df = df.withColumn('median_income', F.col('median_income').cast(FloatType()))
df = df.withColumn('median_house_value', F.col('median_house_value').cast(FloatType()))

In [9]:
df.dtypes

[('longitude', 'float'),
 ('latitude', 'float'),
 ('housing_median_age', 'int'),
 ('total_rooms', 'int'),
 ('total_bedrooms', 'int'),
 ('population', 'int'),
 ('households', 'int'),
 ('median_income', 'float'),
 ('median_house_value', 'float'),
 ('ocean_proximity', 'string')]

In [10]:
# разделим на тест и трейн
train_data, test_data = df.randomSplit([.8,.2], seed=RANDOM_SEED)
print(train_data.count(), test_data.count()) 

16515 4125


In [11]:
categorical_cols = ['ocean_proximity']
numerical_cols  = ["longitude", "latitude", "housing_median_age", 
                   "total_rooms", "total_bedrooms", "population", 
                   "households", "median_income"]
target = "median_house_value" 

In [12]:
indexer = StringIndexer(inputCols=categorical_cols, 
                        outputCols=[c+'_idx' for c in categorical_cols]) 
fitter = indexer.fit(train_data)
train_data = fitter.transform(train_data)
test_data = fitter.transform(test_data)

cols = [c for c in df.columns for i in categorical_cols if (c.startswith(i))]
train_data.select(cols).show(3) 

+---------------+
|ocean_proximity|
+---------------+
|     NEAR OCEAN|
|     NEAR OCEAN|
|     NEAR OCEAN|
+---------------+
only showing top 3 rows



<font color='blue'><b>Комментарий ревьюера: </b></font> ❌\
<font color='red'>Кодирование и Стандартизацию нужно проводить после разделения на выборки трансформерами обучеными на обучающих данных.</font>

<font color='purple'><b>Комментарий студента</b></font>\
<font color=#4B0082><b>Сделал. Перенёс разделение выше и выполнил трансформации через данные трейна.</b></font>

<font color='blue'><b>Комментарий ревьюера: </b></font> ⚠️\
<font color='blue'>Можно использовать [Pipeline](https://habr.com/ru/post/522296/)</font>

<font color='purple'><b>Комментарий студента</b></font>\
<font color=#4B0082><b>Да, всегда использую пайплайн в ml проектах. Но в теории пайпалайном, почему то, назвали простую последовательность преобразований. Статью прочёл, она информативная и с примерами - буду использовать.</b></font>

In [13]:
encoder = OneHotEncoder(inputCols=[c+'_idx' for c in categorical_cols],
                        outputCols=[c+'_ohe' for c in categorical_cols])
fitter = encoder.fit(train_data)
train_data = fitter.transform(train_data)
test_data = fitter.transform(test_data)

cols = [c for c in df.columns for i in categorical_cols if (c.startswith(i))]
train_data.select(cols).show(3) 

+---------------+
|ocean_proximity|
+---------------+
|     NEAR OCEAN|
|     NEAR OCEAN|
|     NEAR OCEAN|
+---------------+
only showing top 3 rows



<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️ <br>
<font color='green'>Верно, что используешь OHE.</font>

In [14]:
categorical_assembler = \
        VectorAssembler(inputCols=[c+'_ohe' for c in categorical_cols],
                                        outputCol="categorical_features")
train_data = categorical_assembler.transform(train_data)
test_data = categorical_assembler.transform(test_data)

In [15]:
numerical_assembler = VectorAssembler(inputCols=numerical_cols, outputCol="numerical_features")
train_data = numerical_assembler.transform(train_data)
test_data = numerical_assembler.transform(test_data)

In [16]:
standardScaler = StandardScaler(inputCol='numerical_features', outputCol="numerical_features_scaled")
fitter = standardScaler.fit(train_data)
train_data = fitter.transform(train_data)
test_data = fitter.transform(test_data)

In [17]:
print(df.columns)

['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity']


In [18]:
all_features = ['categorical_features','numerical_features_scaled']

final_assembler = VectorAssembler(inputCols=all_features, 
                                  outputCol="features") 
train_data = final_assembler.transform(train_data)
test_data = final_assembler.transform(test_data)

train_data.select(all_features).show(3) 

+--------------------+-------------------------+
|categorical_features|numerical_features_scaled|
+--------------------+-------------------------+
|       (4,[2],[1.0])|     [-62.087394450650...|
|       (4,[2],[1.0])|     [-62.062431961328...|
|       (4,[2],[1.0])|     [-62.062431961328...|
+--------------------+-------------------------+
only showing top 3 rows



# Обучение моделей

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Здорово , что у нас есть выборка для тестов!</font>

In [19]:
lr1 = LinearRegression(labelCol=target, featuresCol='features')

model1 = lr1.fit(train_data) 

23/07/12 12:06:20 WARN Instrumentation: [ed8bd6d4] regParam is zero, which might cause numerical instability and overfitting.
23/07/12 12:06:20 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/07/12 12:06:20 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/07/12 12:06:21 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/07/12 12:06:21 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [20]:
predictions1 = model1.transform(test_data)

predictedLabes1 = predictions1.select(target, "prediction")
predictedLabes1.show()

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|           79000.0|184784.96457589697|
|           70500.0| 145578.4707786031|
|           64600.0| 134252.9767572144|
|           81300.0|153131.19892569445|
|           75500.0| 160162.9521834054|
|           85100.0| 180208.6472232798|
|           85600.0| 189570.5222724015|
|           82000.0|201416.83746790187|
|           90600.0|  181322.250163408|
|           74100.0| 150687.3736915365|
|           69500.0|114835.50782892341|
|           90000.0|210354.13294354128|
|           86400.0|194381.71901026508|
|           76800.0|149416.36199384276|
|          128100.0|222344.05520341126|
|           92800.0| 210969.6577821509|
|           94800.0| 226143.3820285434|
|           86900.0| 153577.4737201701|
|           92600.0|170699.81702103047|
|           78400.0|124357.68216068391|
+------------------+------------------+
only showing top 20 rows



<font color='blue'><b>Комментарий ревьюера : </b></font> ✔️\
<font color='green'> 👍</font>

In [21]:
lr2 = LinearRegression(labelCol=target, featuresCol='numerical_features_scaled')

model2 = lr2.fit(train_data) 

23/07/12 12:06:23 WARN Instrumentation: [05c52006] regParam is zero, which might cause numerical instability and overfitting.


In [22]:
predictions2 = model2.transform(test_data)

predictedLabes2 = predictions2.select(target, "prediction")
predictedLabes2.show()

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|           79000.0| 155887.3883871357|
|           70500.0|112091.12482968485|
|           64600.0|100967.52679055277|
|           81300.0|119406.22399751004|
|           75500.0|128645.12284473097|
|           85100.0|150878.96364618186|
|           85600.0|157183.22953275312|
|           82000.0|169627.07351799076|
|           90600.0|148888.61146485852|
|           74100.0|119243.48233028268|
|           69500.0|62517.699619129766|
|           90000.0| 177745.4250109787|
|           86400.0|161243.12849511765|
|           76800.0|117401.79133779695|
|          128100.0|192613.94207159244|
|           92800.0| 178012.4575115433|
|           94800.0|207341.23763665278|
|           86900.0|119359.37586823804|
|           92600.0| 133119.0467339009|
|           78400.0| 85214.49613918317|
+------------------+------------------+
only showing top 20 rows



<font color='blue'><b>Комментарий ревьюера : </b></font> ✔️\
<font color='green'> 👍</font>

# Анализ результатов

In [23]:
rmse1  = RegressionEvaluator(
    labelCol=target, metricName='rmse').evaluate(predictions1)
mae1 = RegressionEvaluator(
    labelCol=target, metricName='mae').evaluate(predictions1)
r21 = RegressionEvaluator(
    labelCol=target, metricName='r2').evaluate(predictions1)

print('RMSE модели для целого датасета: ', rmse1)
print('MAE модели для целого датасета: ', mae1) 
print('R2 модели для целого датасета: ', r21) 

RMSE модели для целого датасета:  68515.87755589013
MAE модели для целого датасета:  50021.948676277134
R2 модели для целого датасета:  0.6454595655138406


In [24]:
rmse2  = RegressionEvaluator(
    labelCol=target, metricName='rmse').evaluate(predictions2)
mae2 = RegressionEvaluator(
    labelCol=target, metricName='mae').evaluate(predictions2)
r22 = RegressionEvaluator(
    labelCol=target, metricName='r2').evaluate(predictions2)

print('RMSE модели без категориальных параметров: ', rmse2)
print('MAE модели без категориальных параметров: ', mae2) 
print('R2 модели без категориальных параметров: ', r22) 

RMSE модели без категориальных параметров:  69145.41652885452
MAE модели без категориальных параметров:  50895.57364243959
R2 модели без категориальных параметров:  0.6389144428155036


<font color='blue'><b>Комментарий ревьюера :</b></font> ✔️\
<font color='green'>Тестирование проведено верно.</font>

ВЫВОД: Линейный модели показывают низкую точность в данной задаче. Категориальный столбец немного портит результат для линейной модели. В данной задаче требуется уделить больше времени на предобработку данных и обучение более тяжеловесных моделей.

In [25]:
spark.stop()

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Очень приятно видеть вывод в конце проекта!\
Приведены ответы на главные вопросы проекта.</font>

<font color='blue'><b>Итоговый комментарий ревьюера</b></font>\
<font color='green'> Александр, хороший проект получился!
Большое спасибо за проделанную работу. Видно, что приложено много усилий.
</font>

<font color='blue'>Что нужно исправить:</font>
<ul><font color='red'>Поправь работу с данными (toPandas только для агрегированных данных).</font></ul>
<ul><font color='red'>Заполни пропуски, выбрав значения по своему усмотрению.</font></ul>
<ul><font color='red'>Стандартизация и Кодировка для train и test по отдельности обработчиком обученным на train.</font></ul>

<font color='darkorange'>Не забывай останавливать сессию `spark.stop()`</font>

<font color='blue'><b>Жду новую версию проекта :)</b></font>